In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm
import random, os
import pickle


# Set environment variables for reproducibility and safety
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import precision_score, recall_score, accuracy_score

# 1. Configuration & Seeding
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

In [ ]:
NAME = 'book' 
INTERACTION_PATH = f'./data/{NAME}_processed_interactions.csv'
GRAPH_PATH = f'./data/{NAME}_processed_graph.csv'

# Hyperparameters
EMBEDDING_DIM = 64
HIDDEN_DIM = 128
LR = 0.001
MAX_STEPS = 3
GAMMA = 0.99 # Discount factor


### 1.1 Split Data

In [ ]:
def load_and_split_data():
    print(">>> [Step 1] Loading and Splitting Data...")
    
    # 1. Load Static KG (Tri thức nền)
    static_kg = pd.read_csv(GRAPH_PATH)
    print(f"Loaded Static KG: {len(static_kg)} edges")
    
    # 2. Load Interactions (Hành vi người dùng)
    interactions = pd.read_csv(INTERACTION_PATH)
    print(f"Loaded Interactions: {len(interactions)} rows")
    # Xử lý thời gian
    interactions['timestamp'] = pd.to_datetime(interactions['timestamp'])
    interactions = interactions.sort_values(by=['user_id', 'timestamp'])
    # Tạo relation_id giả định cho tương tác (nếu chưa có)
    # Ví dụ: gán toàn bộ tương tác là loại quan hệ có ID = -1 hoặc một ID đặc biệt
    # Ở đây ta chỉ cần nó khác với relation_id trong Static KG
    max_kg_relation = static_kg['relation_id'].max()
    INTERACT_REL_ID = max_kg_relation + 1
    interactions['relation_id'] = INTERACT_REL_ID 
    
    # Đổi tên cột cho khớp logic đồ thị: user -> head, item -> tail
    interactions = interactions.rename(columns={'user_id': 'head_id', 'entity_id': 'tail_id'})
    train_data = []
    val_data = []
    test_data = []
    # 3. Chia tập theo User (Chronological)
    grouped = interactions.groupby('head_id')
    
    for user, group in tqdm(grouped, desc="Splitting per user"):
        n = len(group)
        if n < 3: 
            train_data.append(group)
            continue
            
        train_end = int(n * 0.8)
        val_end = int(n * 0.9)
        
        train_data.append(group.iloc[:train_end])
        val_data.append(group.iloc[train_end:val_end])
        test_data.append(group.iloc[val_end:])
    train_df = pd.concat(train_data)
    val_df = pd.concat(val_data)
    test_df = pd.concat(test_data)
    
    print(f"Train/Val/Test sizes: {len(train_df)} / {len(val_df)} / {len(test_df)}")
    return static_kg, train_df, val_df, test_df

In [ ]:
class KGEnvironment:
    def __init__(self, static_kg, train_interactions, embedding_dim):
        with open('../04. RL/pickle/book_transE_embeddings_2026-02-14_16-43-39.pkl', 'rb') as f:
            emb_data = pickle.load(f)
        # Load pretrained weights
        # Lưu ý: Cần map đúng index từ emb_data sang index của graph hiện tại
        # (Ở đây giả sử index khớp hoặc bạn cần viết hàm map lại)
        pretrained_emb = torch.tensor(emb_data['entity_embeddings'])
        self.node_embeds = nn.Embedding.from_pretrained(pretrained_emb, freeze=False) 

        self.graph = nx.MultiDiGraph()
        
        # Build Graph
        all_edges = pd.concat([
            static_kg[['head_id', 'relation_id', 'tail_id']], 
            train_interactions[['head_id', 'relation_id', 'tail_id']]
        ])
        
        for _, row in tqdm(all_edges.iterrows(), total=len(all_edges), desc="Building Graph"):
            self.graph.add_edge(row['head_id'], row['tail_id'], relation=row['relation_id'])
            
        self.nodes = list(self.graph.nodes())
        
        # --- Khởi tạo Embedding ngẫu nhiên cho Node & Relation ---
        # (Trong thực tế nên dùng Pre-trained TransE/Bert để tốt hơn)
        self.max_node_id = max(self.nodes)
        self.max_rel_id = all_edges['relation_id'].max()
        
        # print("Initializing Embeddings...")
        # self.node_embeds = nn.Embedding(self.max_node_id + 1000, embedding_dim) # +1000 buffer
        self.rel_embeds = nn.Embedding(self.max_rel_id + 1000, embedding_dim)
        
    def get_state(self, node_id, hop_k):
        # Trạng thái hiện tại đơn giản là Embedding của Node đó (có thể concat thêm hop)
        # Convert node_id to tensor
        node_idx = torch.tensor([node_id], dtype=torch.long)
        return self.node_embeds(node_idx) 
        
    def get_valid_actions(self, curr_node):
        if not self.graph.has_node(curr_node):
            return [], [], []
        
        neighbors = []
        relations = []
        next_node_ids = []
        
        for u, v, attr in self.graph.out_edges(curr_node, data=True):
            neighbors.append(v)
            relations.append(attr['relation'])
            next_node_ids.append(v)
            
        return neighbors, relations, next_node_ids

In [ ]:
# ==========================================
# 4. POLICY NETWORK (The Brain)
# ==========================================
class PolicyNetwork(nn.Module):
    def __init__(self, state_dim, hidden_dim):
        super(PolicyNetwork, self).__init__()
        # Input: [State_Emb; Relation_Emb; Next_Node_Emb]
        # Output: Score scalar
        self.fc1 = nn.Linear(state_dim * 3, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 1) # Điểm số cho hành động
        self.relu = nn.ReLU()
        
    def forward(self, state_emb, action_embs):
        # state_emb: (1, dim)
        # action_embs (Neighbors): (num_neighbors, dim * 2) gồm [Rel; Next_Node]
        
        # Expand state to match neighbors count
        num_actions = action_embs.shape[0]
        curr_state = state_emb.repeat(num_actions, 1) # (num_neighbors, dim)
        
        # Concat: [Current State, Relation, Next Node]
        x = torch.cat([curr_state, action_embs], dim=1) # (num_neighbors, dim*3)
        
        x = self.relu(self.fc1(x))
        scores = self.fc2(x) # (num_neighbors, 1)
        
        # Softmax để ra xác suất chọn từng neighbor
        probs = F.softmax(scores.view(-1), dim=0)
        return probs

In [ ]:
# ==========================================
# 5. TRAINING LOOP (REINFORCE)
# ==========================================
def train(env, policy_net, train_df, episodes=50):
    optimizer = optim.Adam(policy_net.parameters(), lr=LR)
    
    print("\n>>> Start Training...")
    
    for episode in range(episodes):
        total_reward = 0
        total_loss = 0
        
        # Sample batch user để train cho nhanh (Stochastic)
        batch_samples = train_df.sample(n=256) 
        
        optimizer.zero_grad()
        batch_loss = 0
        
        for _, row in batch_samples.iterrows():
            curr_node = row['head_id']
            target_node = row['tail_id']
            
            episode_log_probs = []
            rewards = []
            
            # --- Walking ---
            done = False
            for step in range(MAX_STEPS):
                # 1. Get State
                state = env.get_state(curr_node, step) # (1, dim)
                
                # 2. Get Valid Actions (Neighbors)
                neighbors, rels, neighbor_ids = env.get_valid_actions(curr_node)
                
                if len(neighbors) == 0:
                    break # Dead end
                
                # 3. Tạo Embedding cho Actions để đưa vào mạng
                neighbor_tensor = torch.tensor(neighbor_ids, dtype=torch.long)
                rel_tensor = torch.tensor(rels, dtype=torch.long)
                
                neigh_embeds = env.node_embeds(neighbor_tensor)
                rel_embeds = env.rel_embeds(rel_tensor)
                action_features = torch.cat([rel_embeds, neigh_embeds], dim=1) # (num_neighbors, dim*2)
                
                # 4. Agent chọn hành động
                probs = policy_net(state, action_features)
                
                # Sampling action dựa trên xác suất (Exploration)
                dist = torch.distributions.Categorical(probs)
                action_idx = dist.sample()
                log_prob = dist.log_prob(action_idx)
                
                episode_log_probs.append(log_prob)
                
                # 5. Execute Action
                next_node = neighbors[action_idx.item()]
                curr_node = next_node
                
                # 6. Check Reward
                if curr_node == target_node:
                    rewards.append(1.0) # Tìm thấy!
                    done = True
                    break
                else:
                    rewards.append(0.0) # Chưa thấy
            
            # --- Tính Loss (Policy Gradient) ---
            # Nếu tìm thấy đích ở bước cuối, phần thưởng lan truyền ngược lại
            # Discounted Return
            R = 0
            returns = []
            for r in reversed(rewards):
                R = r + GAMMA * R
                returns.insert(0, R)
            
            if done: # Chỉ học nếu tìm thấy đích (hoặc có thể phạt nhẹ nếu không tìm thấy)
                total_reward += 1
                for log_prob, R in zip(episode_log_probs, returns):
                    batch_loss -= log_prob * R # Gradient Ascent -> Minimize Negative Reward
        
        # Update Weights
        if batch_loss != 0:
            batch_loss.backward()
            optimizer.step()
            
        if (episode+1) % 5 == 0:
            print(f"Episode {episode+1}/{episodes} | Hit Success: {total_reward}/256 | Batch Loss: {batch_loss:.4f}")
            

In [ ]:
def evaluate(env, policy_net, test_df, top_k=10):
    print("\n>>> Start Evaluation...")
    policy_net.eval() # Chuyển sang chế độ đánh giá (tắt dropout nếu có)
    
    hits = 0
    total_samples = 0
    
    # Lấy mẫu ngẫu nhiên từ Test set để chạy cho nhanh (hoặc chạy hết nếu muốn chính xác)
    test_samples = test_df.sample(n=100) if len(test_df) > 100 else test_df
    
    with torch.no_grad(): # Tắt tính toán gradient để tiết kiệm bộ nhớ
        for _, row in tqdm(test_samples.iterrows(), total=len(test_samples)):
            curr_node = row['head_id']
            target_true = row['tail_id']
            
            # --- Beam Search (Giản lược: Greedy Best-First) ---
            # Để đơn giản, ta cho Agent chọn top-1 đường đi tốt nhất
            
            for step in range(MAX_STEPS):
                state = env.get_state(curr_node, step)
                neighbors, rels, neighbor_ids = env.get_valid_actions(curr_node)
                
                if not neighbors: break 
                
                # Chuẩn bị input cho mạng
                neighbor_tensor = torch.tensor(neighbor_ids, dtype=torch.long)
                rel_tensor = torch.tensor(rels, dtype=torch.long)
                neigh_embeds = env.node_embeds(neighbor_tensor)
                rel_embeds = env.rel_embeds(rel_tensor)
                action_features = torch.cat([rel_embeds, neigh_embeds], dim=1)
                
                # Dự đoán xác suất
                probs = policy_net(state, action_features)
                
                # --- KHÁC BIỆT: Chọn bước đi tốt nhất (Argmax) ---
                # Thay vì random sample, ta chọn nước đi có xác suất cao nhất
                best_action_idx = torch.argmax(probs).item()
                
                next_node = neighbors[best_action_idx]
                curr_node = next_node
                
                # Kiểm tra xem node hiện tại có phải là đích không
                if curr_node == target_true:
                    hits += 1
                    break
            
            total_samples += 1
            
    # Tính HR@1 (Hit Rate) - Tỉ lệ tìm thấy chính xác Item
    acc = hits / total_samples if total_samples > 0 else 0
    print(f"Evaluation Rank 1 (Exact Match): {acc:.4f}")
    
    return acc

In [ ]:
# ==========================================
# MAIN
# ==========================================
if __name__ == "__main__":
    # 1. Load Data
    static_kg, train_df, val_df, test_df = load_and_split_data()
    
    # 2. Init Env (Cần Embedding cho Node/Rel)
    env = KGEnvironment(static_kg, train_df, embedding_dim=EMBEDDING_DIM)
    
    # 3. Init Agent
    policy_net = PolicyNetwork(state_dim=EMBEDDING_DIM, hidden_dim=HIDDEN_DIM)
    
    # 4. Train
    train(env, policy_net, train_df, episodes=50000) # Tăng episodes để thấy reward tăng

    evaluate(env, policy_net, test_df)